In [1]:
import torch
from torch import nn
from torch.utils import data
from torchvision import datasets, transforms, models
from tqdm.notebook import tqdm
import numpy as np
from sklearn.model_selection import train_test_split
from copy import deepcopy

from nn_extrapolation import AcceleratedSGD

In [2]:
val_loss_fn = nn.NLLLoss(reduction="sum")

def validation(model, loader):
    ok = 0
    loss_sum = 0
    total = 0
    model.eval()
    with torch.no_grad():
        for x, y in tqdm(loader):
            x = x.cuda()
            y = y.cuda()
            out = model(x)
            loss_sum += val_loss_fn(out, y).item()
            preds = out.argmax(1)
            ok += (preds == y).sum().item()
            total += len(y)
    return ok / total, loss_sum / total

def train_epoch(loss_log):
    model.train()
    for x, y in tqdm(train_loader):
        x = x.cuda()
        y = y.cuda()
        optimizer.zero_grad()
        out = model(x)
        loss = loss_fn(out, y)
        loss_log += list(loss.flatten().cpu().detach().numpy())
        loss.backward()
        optimizer.step()

In [3]:
augmentation = transforms.RandomAffine(10, scale=(0.9, 1.1), translate=(0.2, 0.2))
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

transform = transforms.Compose([
    transforms.ToTensor(),
    normalize,
])
train_ds = datasets.CIFAR10("/tmp/i291318/CIFAR", 
                            download=True, train=True, transform=transforms.Compose([augmentation, transform]))
valid_ds = datasets.CIFAR10("/tmp/i291318/CIFAR", download=True, train=True, transform=transform)
test_ds = datasets.CIFAR10("/tmp/i291318/CIFAR", download=True, train=False, transform=transform)

train_indices, valid_indices = train_test_split(np.arange(len(train_ds)), test_size=0.2)
train_ds = data.Subset(train_ds, train_indices)
valid_ds = data.Subset(valid_ds, valid_indices)

train_loader = data.DataLoader(train_ds, batch_size=128, shuffle=True, num_workers=4)
valid_loader = data.DataLoader(valid_ds, batch_size=128, shuffle=False, num_workers=4)
test_loader = data.DataLoader(test_ds, batch_size=128, shuffle=False, num_workers=4)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [4]:
model = models.resnet18(pretrained=False)
model.fc = nn.Sequential(
    nn.Linear(512, 10),
    nn.LogSoftmax(-1)
)
model.cuda()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [5]:
validation(model, valid_loader)

(0.1005, 2.3273343299865723)

In [6]:
log_file = open("resnet_log_augmentation_splits.txt", "w")

## All parameters separate

In [7]:
groups = [{"params": [param]} for param in model.parameters()]
optimizer = AcceleratedSGD(groups, 1e-1, k=10, momentum=0.9, weight_decay=1e-5)
loss_fn = nn.NLLLoss()

In [8]:
epochs = 30

for epoch in range(epochs):
    print("Epoch", epoch+1)
    loss_log = []
    train_epoch(loss_log)
    print(f"Training loss: {np.mean(loss_log):.4f}")
    optimizer.finish_epoch()
    val_acc, val_loss = validation(model, valid_loader)
    print(f"Validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}")
    print("Epoch", epoch+1, 
          f"Training loss: {np.mean(loss_log):.4f}, validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}",
          file=log_file, flush=True
         )

Epoch 1



Training loss: 2.3596



Validation accuracy: 0.3244, validation loss: 2.2152
Epoch 2



Training loss: 1.7322



Validation accuracy: 0.4083, validation loss: 1.5957
Epoch 3


...


Validation accuracy: 0.7706, validation loss: 0.6866
Epoch 29



Training loss: 0.6461



Validation accuracy: 0.7631, validation loss: 0.6956
Epoch 30



Training loss: 0.6263



Validation accuracy: 0.7601, validation loss: 0.7368


In [9]:
train_score = validation(model, train_loader)
valid_score = validation(model, valid_loader)
print("Train:", train_score)
print("Valid:", valid_score)
print("Train:", train_score, flush=True, file=log_file)
print("Valid:", valid_score, flush=True, file=log_file)


Train: (0.781825, 0.6188753798484802)
Valid: (0.7601, 0.7368492457389831)


In [10]:
model_acc = deepcopy(model)
optimizer.accelerate()
groups = [[param] for param in model_acc.parameters()]
optimizer.store_parameters(groups)

In [11]:
model_acc.cuda()
train_score = validation(model_acc, train_loader)
valid_score = validation(model_acc, valid_loader)
model_acc.cpu()
model.cuda()

print("Train:", train_score)
print("Valid:", valid_score)
print("Train:", train_score, flush=True, file=log_file)
print("Valid:", valid_score, flush=True, file=log_file)


Train: (0.737875, 0.7698297311782837)
Valid: (0.7253, 0.8823400542259217)


In [12]:
optimizer.param_groups[0]["lr"] = 1e-2

In [13]:
epochs = 30

for epoch in range(epochs):
    print("Epoch", epoch+1)
    loss_log = []
    train_epoch(loss_log)
    print(f"Training loss: {np.mean(loss_log):.4f}")
    optimizer.finish_epoch()
    val_acc, val_loss = validation(model, valid_loader)
    print(f"Validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}")
    print("Epoch", epoch+1, 
          f"Training loss: {np.mean(loss_log):.4f}, validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}",
          file=log_file, flush=True
         )

Epoch 1



Training loss: 0.6007



Validation accuracy: 0.7711, validation loss: 0.6972
Epoch 2



Training loss: 0.5893



Validation accuracy: 0.7779, validation loss: 0.6621
Epoch 3


...


Validation accuracy: 0.7969, validation loss: 0.6415
Epoch 29



Training loss: 0.3881



Validation accuracy: 0.7932, validation loss: 0.6546
Epoch 30



Training loss: 0.3838



Validation accuracy: 0.7870, validation loss: 0.7275


In [14]:
train_score = validation(model, train_loader)
valid_score = validation(model, valid_loader)
print("Train:", train_score)
print("Valid:", valid_score)
print("Train:", train_score, flush=True, file=log_file)
print("Valid:", valid_score, flush=True, file=log_file)


Train: (0.86995, 0.3662867865562439)
Valid: (0.787, 0.727545150566101)


In [15]:
model_acc = deepcopy(model)
optimizer.accelerate()
groups = [[param] for param in model_acc.parameters()]
optimizer.store_parameters(groups)

In [16]:
model_acc.cuda()
train_score = validation(model_acc, train_loader)
valid_score = validation(model_acc, valid_loader)
model_acc.cpu()
model.cuda()

print("Train:", train_score)
print("Valid:", valid_score)
print("Train:", train_score, flush=True, file=log_file)
print("Valid:", valid_score, flush=True, file=log_file)


Train: (0.857225, 0.40191869921684265)
Valid: (0.7889, 0.6831509204864502)


In [17]:
optimizer.param_groups[0]["lr"] = 1e-3

In [18]:
epochs = 30

for epoch in range(epochs):
    print("Epoch", epoch+1)
    loss_log = []
    train_epoch(loss_log)
    print(f"Training loss: {np.mean(loss_log):.4f}")
    optimizer.finish_epoch()
    val_acc, val_loss = validation(model, valid_loader)
    print(f"Validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}")
    print("Epoch", epoch+1, 
          f"Training loss: {np.mean(loss_log):.4f}, validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}",
          file=log_file, flush=True
         )

Epoch 1



Training loss: 0.3812



Validation accuracy: 0.7991, validation loss: 0.6706
Epoch 2



Training loss: 0.3752



Validation accuracy: 0.7948, validation loss: 0.6720
Epoch 3


...


Validation accuracy: 0.8005, validation loss: 0.7403
Epoch 29



Training loss: 0.2719



Validation accuracy: 0.8042, validation loss: 0.6993
Epoch 30



Training loss: 0.2741



Validation accuracy: 0.8013, validation loss: 0.7104


In [19]:
train_score = validation(model, train_loader)
valid_score = validation(model, valid_loader)
print("Train:", train_score)
print("Valid:", valid_score)
print("Train:", train_score, flush=True, file=log_file)
print("Valid:", valid_score, flush=True, file=log_file)


Train: (0.909175, 0.2521676215648651)
Valid: (0.8013, 0.7103771095275879)


In [20]:
model_acc = deepcopy(model)
optimizer.accelerate()
groups = [[param] for param in model_acc.parameters()]
optimizer.store_parameters(groups)

In [21]:
model_acc.cuda()
train_score = validation(model_acc, train_loader)
valid_score = validation(model_acc, valid_loader)
model_acc.cpu()
model.cuda()

print("Train:", train_score)
print("Valid:", valid_score)
print("Train:", train_score, flush=True, file=log_file)
print("Valid:", valid_score, flush=True, file=log_file)


Train: (0.92085, 0.2278799115896225)
Valid: (0.8104, 0.6527196309089661)


## All layers separate

In [22]:
model = models.resnet18(pretrained=False)
model.fc = nn.Sequential(
    nn.Linear(512, 10),
    nn.LogSoftmax(-1)
)
model.cuda()
None

In [23]:
validation(model, valid_loader)

(0.1151, 2.3191361389160154)

In [24]:
log_file = open("resnet_log_augmentation_layer_splits.txt", "w")

In [25]:
def module_layers(module):
    has_children = False
    for child in module.children():
        has_children = True
        yield from module_layers(child)
    if not has_children:
        yield module

In [26]:
groups = []
for layer in module_layers(model):
    params = list(layer.parameters())
    if params:
        groups.append({"params": params})
optimizer = AcceleratedSGD(groups, 1e-1, k=10, momentum=0.9, weight_decay=1e-5)
loss_fn = nn.NLLLoss()

In [27]:
epochs = 30

for epoch in range(epochs):
    print("Epoch", epoch+1)
    loss_log = []
    train_epoch(loss_log)
    print(f"Training loss: {np.mean(loss_log):.4f}")
    optimizer.finish_epoch()
    val_acc, val_loss = validation(model, valid_loader)
    print(f"Validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}")
    print("Epoch", epoch+1, 
          f"Training loss: {np.mean(loss_log):.4f}, validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}",
          file=log_file, flush=True
         )

Epoch 1



Training loss: 2.4104



Validation accuracy: 0.3432, validation loss: 1.7926
Epoch 2



Training loss: 1.7549



Validation accuracy: 0.4316, validation loss: 1.5536
Epoch 3


...


Validation accuracy: 0.7574, validation loss: 0.7000
Epoch 29



Training loss: 0.6528



Validation accuracy: 0.7643, validation loss: 0.6990
Epoch 30



Training loss: 0.6427



Validation accuracy: 0.7643, validation loss: 0.6988


In [28]:
train_score = validation(model, train_loader)
valid_score = validation(model, valid_loader)
print("Train:", train_score)
print("Valid:", valid_score)
print("Train:", train_score, flush=True, file=log_file)
print("Valid:", valid_score, flush=True, file=log_file)


Train: (0.7829, 0.6147592720031738)
Valid: (0.7643, 0.6987689692497253)


In [29]:
model_acc = deepcopy(model)
optimizer.accelerate()
groups = []
for layer in module_layers(model_acc):
    params = list(layer.parameters())
    if params:
        groups.append(params)
optimizer.store_parameters(groups)

In [30]:
model_acc.cuda()
train_score = validation(model_acc, train_loader)
valid_score = validation(model_acc, valid_loader)
model_acc.cpu()
model.cuda()

print("Train:", train_score)
print("Valid:", valid_score)
print("Train:", train_score, flush=True, file=log_file)
print("Valid:", valid_score, flush=True, file=log_file)


Train: (0.783325, 0.616992482662201)
Valid: (0.765, 0.6951740602493286)


In [31]:
optimizer.param_groups[0]["lr"] = 1e-2

In [32]:
epochs = 30

for epoch in range(epochs):
    print("Epoch", epoch+1)
    loss_log = []
    train_epoch(loss_log)
    print(f"Training loss: {np.mean(loss_log):.4f}")
    optimizer.finish_epoch()
    val_acc, val_loss = validation(model, valid_loader)
    print(f"Validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}")
    print("Epoch", epoch+1, 
          f"Training loss: {np.mean(loss_log):.4f}, validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}",
          file=log_file, flush=True
         )

Epoch 1



Training loss: 0.6077



Validation accuracy: 0.7730, validation loss: 0.6776
Epoch 2



Training loss: 0.5969



Validation accuracy: 0.7824, validation loss: 0.6573
Epoch 3


...


Validation accuracy: 0.7883, validation loss: 0.7162
Epoch 29



Training loss: 0.3898



Validation accuracy: 0.7930, validation loss: 0.6513
Epoch 30



Training loss: 0.3861



Validation accuracy: 0.7832, validation loss: 0.7287


In [33]:
train_score = validation(model, train_loader)
valid_score = validation(model, valid_loader)
print("Train:", train_score)
print("Valid:", valid_score)
print("Train:", train_score, flush=True, file=log_file)
print("Valid:", valid_score, flush=True, file=log_file)


Train: (0.8637, 0.38939660983085633)
Valid: (0.7832, 0.7287291321754455)


In [34]:
model_acc = deepcopy(model)
optimizer.accelerate()
groups = []
for layer in module_layers(model_acc):
    params = list(layer.parameters())
    if params:
        groups.append(params)
optimizer.store_parameters(groups)

In [35]:
model_acc.cuda()
train_score = validation(model_acc, train_loader)
valid_score = validation(model_acc, valid_loader)
model_acc.cpu()
model.cuda()

print("Train:", train_score)
print("Valid:", valid_score)
print("Train:", train_score, flush=True, file=log_file)
print("Valid:", valid_score, flush=True, file=log_file)


Train: (0.87685, 0.34487105107307436)
Valid: (0.8047, 0.6410033418655395)


In [36]:
optimizer.param_groups[0]["lr"] = 1e-3

In [37]:
epochs = 30

for epoch in range(epochs):
    print("Epoch", epoch+1)
    loss_log = []
    train_epoch(loss_log)
    print(f"Training loss: {np.mean(loss_log):.4f}")
    optimizer.finish_epoch()
    val_acc, val_loss = validation(model, valid_loader)
    print(f"Validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}")
    print("Epoch", epoch+1, 
          f"Training loss: {np.mean(loss_log):.4f}, validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}",
          file=log_file, flush=True
         )

Epoch 1



Training loss: 0.3894



Validation accuracy: 0.7988, validation loss: 0.6751
Epoch 2



Training loss: 0.3792



Validation accuracy: 0.7956, validation loss: 0.6476
Epoch 3


...


Validation accuracy: 0.8041, validation loss: 0.7137
Epoch 29



Training loss: 0.2808



Validation accuracy: 0.8061, validation loss: 0.6778
Epoch 30



Training loss: 0.2741



Validation accuracy: 0.7992, validation loss: 0.7367


In [38]:
train_score = validation(model, train_loader)
valid_score = validation(model, valid_loader)
print("Train:", train_score)
print("Valid:", valid_score)
print("Train:", train_score, flush=True, file=log_file)
print("Valid:", valid_score, flush=True, file=log_file)


Train: (0.909025, 0.25360990395545957)
Valid: (0.7992, 0.7366648662567139)


In [39]:
model_acc = deepcopy(model)
optimizer.accelerate()
groups = []
for layer in module_layers(model_acc):
    params = list(layer.parameters())
    if params:
        groups.append(params)
optimizer.store_parameters(groups)

In [40]:
model_acc.cuda()
train_score = validation(model_acc, train_loader)
valid_score = validation(model_acc, valid_loader)
model_acc.cpu()
model.cuda()

print("Train:", train_score)
print("Valid:", valid_score)
print("Train:", train_score, flush=True, file=log_file)
print("Valid:", valid_score, flush=True, file=log_file)


Train: (0.923925, 0.21591134462356568)
Valid: (0.8146, 0.6561075517654419)


## Conv and linear separate

In [41]:
model = models.resnet18(pretrained=False)
model.fc = nn.Sequential(
    nn.Linear(512, 10),
    nn.LogSoftmax(-1)
)
model.cuda()
None

In [42]:
validation(model, valid_loader)

(0.0992, 2.328910371017456)

In [43]:
log_file = open("resnet_log_augmentation_conv_linear_splits.txt", "w")

In [44]:
len(list(model.fc.parameters()))

2

In [45]:
params = list(model.parameters())
groups = [
    { "params": params[:-2] },
    { "params": params[-2:] },
]
optimizer = AcceleratedSGD(groups, 1e-1, k=10, momentum=0.9, weight_decay=1e-5)
loss_fn = nn.NLLLoss()

In [46]:
epochs = 30

for epoch in range(epochs):
    print("Epoch", epoch+1)
    loss_log = []
    train_epoch(loss_log)
    print(f"Training loss: {np.mean(loss_log):.4f}")
    optimizer.finish_epoch()
    val_acc, val_loss = validation(model, valid_loader)
    print(f"Validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}")
    print("Epoch", epoch+1, 
          f"Training loss: {np.mean(loss_log):.4f}, validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}",
          file=log_file, flush=True
         )

Epoch 1



Training loss: 2.2642



Validation accuracy: 0.3359, validation loss: 2.4528
Epoch 2



Training loss: 1.6927



Validation accuracy: 0.4025, validation loss: 1.6338
Epoch 3


...


Validation accuracy: 0.7633, validation loss: 0.7074
Epoch 29



Training loss: 0.6148



Validation accuracy: 0.7701, validation loss: 0.6945
Epoch 30



Training loss: 0.6040



Validation accuracy: 0.7806, validation loss: 0.6501


In [47]:
train_score = validation(model, train_loader)
valid_score = validation(model, valid_loader)
print("Train:", train_score)
print("Valid:", valid_score)
print("Train:", train_score, flush=True, file=log_file)
print("Valid:", valid_score, flush=True, file=log_file)


Train: (0.808225, 0.5443021055221557)
Valid: (0.7806, 0.6501237643241883)


In [48]:
model_acc = deepcopy(model)
optimizer.accelerate()
params = list(model_acc.parameters())
groups = [params[:-2], params[-2:]]
optimizer.store_parameters(groups)

In [49]:
model_acc.cuda()
train_score = validation(model_acc, train_loader)
valid_score = validation(model_acc, valid_loader)
model_acc.cpu()
model.cuda()

print("Train:", train_score)
print("Valid:", valid_score)
print("Train:", train_score, flush=True, file=log_file)
print("Valid:", valid_score, flush=True, file=log_file)


Train: (0.7923, 0.5939255303382873)
Valid: (0.7723, 0.6731794701576232)


In [50]:
optimizer.param_groups[0]["lr"] = 1e-2

In [ ]:
epochs = 30

for epoch in range(epochs):
    print("Epoch", epoch+1)
    loss_log = []
    train_epoch(loss_log)
    print(f"Training loss: {np.mean(loss_log):.4f}")
    optimizer.finish_epoch()
    val_acc, val_loss = validation(model, valid_loader)
    print(f"Validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}")
    print("Epoch", epoch+1, 
          f"Training loss: {np.mean(loss_log):.4f}, validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}",
          file=log_file, flush=True
         )

Epoch 1



Training loss: 0.5046



Validation accuracy: 0.7992, validation loss: 0.5930
Epoch 2



Training loss: 0.4780



Validation accuracy: 0.8007, validation loss: 0.6001
Epoch 3


...


Validation accuracy: 0.8026, validation loss: 0.6013
Epoch 6



Training loss: 0.4365



Validation accuracy: 0.8053, validation loss: 0.6051
Epoch 7



Training loss: 0.4355



Validation accuracy: 0.8039, validation loss: 0.5948
Epoch 8


In [ ]:
train_score = validation(model, train_loader)
valid_score = validation(model, valid_loader)
print("Train:", train_score)
print("Valid:", valid_score)
print("Train:", train_score, flush=True, file=log_file)
print("Valid:", valid_score, flush=True, file=log_file)

In [ ]:
model_acc = deepcopy(model)
optimizer.accelerate()
params = list(model_acc.parameters())
groups = [params[:-2], params[-2:]]
optimizer.store_parameters(groups)

In [ ]:
model_acc.cuda()
train_score = validation(model_acc, train_loader)
valid_score = validation(model_acc, valid_loader)
model_acc.cpu()
model.cuda()

print("Train:", train_score)
print("Valid:", valid_score)
print("Train:", train_score, flush=True, file=log_file)
print("Valid:", valid_score, flush=True, file=log_file)

In [ ]:
optimizer.param_groups[0]["lr"] = 1e-3

In [ ]:
epochs = 30

for epoch in range(epochs):
    print("Epoch", epoch+1)
    loss_log = []
    train_epoch(loss_log)
    print(f"Training loss: {np.mean(loss_log):.4f}")
    optimizer.finish_epoch()
    val_acc, val_loss = validation(model, valid_loader)
    print(f"Validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}")
    print("Epoch", epoch+1, 
          f"Training loss: {np.mean(loss_log):.4f}, validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}",
          file=log_file, flush=True
         )

In [ ]:
train_score = validation(model, train_loader)
valid_score = validation(model, valid_loader)
print("Train:", train_score)
print("Valid:", valid_score)
print("Train:", train_score, flush=True, file=log_file)
print("Valid:", valid_score, flush=True, file=log_file)

In [ ]:
model_acc = deepcopy(model)
optimizer.accelerate()
params = list(model_acc.parameters())
groups = [params[:-2], params[-2:]]
optimizer.store_parameters(groups)

In [ ]:
model_acc.cuda()
train_score = validation(model_acc, train_loader)
valid_score = validation(model_acc, valid_loader)
model_acc.cpu()
model.cuda()

print("Train:", train_score)
print("Valid:", valid_score)
print("Train:", train_score, flush=True, file=log_file)
print("Valid:", valid_score, flush=True, file=log_file)

## Conv and linear separate, only accelerate linear

In [ ]:
model = models.resnet18(pretrained=False)
model.fc = nn.Sequential(
    nn.Linear(512, 10),
    nn.LogSoftmax(-1)
)
model.cuda()
None

In [ ]:
validation(model, valid_loader)

In [ ]:
log_file = open("resnet_log_augmentation_conv_linear_splits_linear_acc.txt", "w")

In [ ]:
len(list(model.fc.parameters()))

In [ ]:
params = list(model.parameters())
groups = [
    { "params": params[:-2], "method": None },
    { "params": params[-2:] },
]
optimizer = AcceleratedSGD(groups, 1e-1, k=10, momentum=0.9, weight_decay=1e-5)
loss_fn = nn.NLLLoss()

In [ ]:
epochs = 30

for epoch in range(epochs):
    print("Epoch", epoch+1)
    loss_log = []
    train_epoch(loss_log)
    print(f"Training loss: {np.mean(loss_log):.4f}")
    optimizer.finish_epoch()
    val_acc, val_loss = validation(model, valid_loader)
    print(f"Validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}")
    print("Epoch", epoch+1, 
          f"Training loss: {np.mean(loss_log):.4f}, validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}",
          file=log_file, flush=True
         )

In [ ]:
train_score = validation(model, train_loader)
valid_score = validation(model, valid_loader)
print("Train:", train_score)
print("Valid:", valid_score)
print("Train:", train_score, flush=True, file=log_file)
print("Valid:", valid_score, flush=True, file=log_file)

In [ ]:
model_acc = deepcopy(model)
optimizer.accelerate()
params = list(model_acc.parameters())
groups = [params[:-2], params[-2:]]
optimizer.store_parameters(groups)

In [ ]:
model_acc.cuda()
train_score = validation(model_acc, train_loader)
valid_score = validation(model_acc, valid_loader)
model_acc.cpu()
model.cuda()

print("Train:", train_score)
print("Valid:", valid_score)
print("Train:", train_score, flush=True, file=log_file)
print("Valid:", valid_score, flush=True, file=log_file)

In [ ]:
optimizer.param_groups[0]["lr"] = 1e-2

In [ ]:
epochs = 30

for epoch in range(epochs):
    print("Epoch", epoch+1)
    loss_log = []
    train_epoch(loss_log)
    print(f"Training loss: {np.mean(loss_log):.4f}")
    optimizer.finish_epoch()
    val_acc, val_loss = validation(model, valid_loader)
    print(f"Validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}")
    print("Epoch", epoch+1, 
          f"Training loss: {np.mean(loss_log):.4f}, validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}",
          file=log_file, flush=True
         )

In [ ]:
train_score = validation(model, train_loader)
valid_score = validation(model, valid_loader)
print("Train:", train_score)
print("Valid:", valid_score)
print("Train:", train_score, flush=True, file=log_file)
print("Valid:", valid_score, flush=True, file=log_file)

In [ ]:
model_acc = deepcopy(model)
optimizer.accelerate()
params = list(model_acc.parameters())
groups = [params[:-2], params[-2:]]
optimizer.store_parameters(groups)

In [ ]:
model_acc.cuda()
train_score = validation(model_acc, train_loader)
valid_score = validation(model_acc, valid_loader)
model_acc.cpu()
model.cuda()

print("Train:", train_score)
print("Valid:", valid_score)
print("Train:", train_score, flush=True, file=log_file)
print("Valid:", valid_score, flush=True, file=log_file)

In [ ]:
optimizer.param_groups[0]["lr"] = 1e-3

In [ ]:
epochs = 30

for epoch in range(epochs):
    print("Epoch", epoch+1)
    loss_log = []
    train_epoch(loss_log)
    print(f"Training loss: {np.mean(loss_log):.4f}")
    optimizer.finish_epoch()
    val_acc, val_loss = validation(model, valid_loader)
    print(f"Validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}")
    print("Epoch", epoch+1, 
          f"Training loss: {np.mean(loss_log):.4f}, validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}",
          file=log_file, flush=True
         )

In [ ]:
train_score = validation(model, train_loader)
valid_score = validation(model, valid_loader)
print("Train:", train_score)
print("Valid:", valid_score)
print("Train:", train_score, flush=True, file=log_file)
print("Valid:", valid_score, flush=True, file=log_file)

In [ ]:
model_acc = deepcopy(model)
optimizer.accelerate()
params = list(model_acc.parameters())
groups = [params[:-2], params[-2:]]
optimizer.store_parameters(groups)

In [ ]:
model_acc.cuda()
train_score = validation(model_acc, train_loader)
valid_score = validation(model_acc, valid_loader)
model_acc.cpu()
model.cuda()

print("Train:", train_score)
print("Valid:", valid_score)
print("Train:", train_score, flush=True, file=log_file)
print("Valid:", valid_score, flush=True, file=log_file)

In [ ]:
sys.exit(0)